In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DV_A3/100k_spotify_songs.csv")

<ipython-input-2-89cce5490565>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/DV_A3/corrected_file_1.csv")


In [ ]:
# df = df.loc[:, ~df.columns.str.contains(r'Unnamed|extra', case=False)]
# print(df)

In [ ]:
threshold = 0.75 * df.shape[1]
df = df[df.isnull().sum(axis=1) < threshold]

In [ ]:
df.to_csv("new.csv",index=False)

In [ ]:
# duplicate_track_ids = df['track_id'].value_counts()[df['track_id'].value_counts() > 1].index
# print(duplicate_track_ids)
# df = df[~df['track_id'].isin(duplicate_track_ids)]

In [ ]:
# print(df.shape)

In [ ]:
# df.to_csv("final_dataset_100k.csv",index=False)

In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.0 MB/s eta 0:00:00


In [ ]:
df_1 = pd.read_csv("/content/most_streamed_spotify_songs_till2022.csv")

In [ ]:
df = pd.read_csv("/content/final_dataset_100k.csv")
merged = pd.read_csv("/content/merged.csv")


In [ ]:
import pandas as pd
from rapidfuzz.fuzz import ratio
from rapidfuzz import process

df['track_artist_tuple'] = list(zip(df['track_name'], df['artists']))

def find_best_matches(df, df_1, track_threshold=66, artist_threshold=66):
    matches = []

    for idx_1, row_1 in df_1.iterrows():
        track_1 = row_1['Track']
        artist_1 = row_1['Artist']

        if (track_1, artist_1) in merged[['Track', 'Artist']].itertuples(index=False, name=None):
            print(f"{track_1} by {artist_1} already in merged")
            continue

        for idx_df, row_df in df.iterrows():
            track_2 = row_df['track_name']
            artist_2 = [artist.strip() for artist in row_df['artists'].split(',')]

            track_score = ratio(track_1, track_2)
            artist_score = max(ratio(artist_1, artist) for artist in artist_2)

            if track_score >= track_threshold and artist_score >= artist_threshold:
                print(artist_score, artist_1, artist_2, track_score, track_1, track_2)
                matches.append({
                    'df_index': idx_df,
                    'df_1_index': idx_1,
                    'track_score': track_score,
                    'artist_score': artist_score
                })

    return matches

matches = find_best_matches(df, df_1)


TOMBOY by (G)I-DLE already in merged
100.0 24kGoldn ['24kGoldn', 'iann dior'] 100.0 Mood (feat. iann dior) Mood (feat. iann dior)
Dancing in My Room by 347aidan already in merged
MEMORIES! by 347aidan already in merged
Youngblood by 5 Seconds of Summer already in merged
Teeth by 5 Seconds of Summer already in merged
GOOBA by 6ix9ine already in merged
Praise The Lord (Da Shine) (feat. Skepta) by A$AP Rocky already in merged
Dancin (feat. Luvli) - Krono Remix by Aaron Smith already in merged
Set Fire to the Rain by Adele already in merged
Rolling in the Deep - Acapella by Adele already in merged
Send My Love (To Your New Lover) by Adele already in merged
Savage by aespa already in merged
Next Level by aespa already in merged
Girls by aespa already in merged
World's Smallest Violin by AJR already in merged
Tera Ban Jaunga by Akhil Sachdeva already in merged
How Far I'll Go - From "Moana" by Alessia Cara already in merged
Little Do You Know by Alex & Sierra already in merged
Girl On Fire b

In [ ]:
print(matches)

# Merge DataFrames based on matches
if matches:
    df_matches = pd.DataFrame(matches)
    # Use indices to align data
    merged_df = pd.merge(
        df.loc[df_matches['df_index']],
        df_1.loc[df_matches['df_1_index']].reset_index(drop=True),
        left_index=True,
        right_index=True,
        suffixes=('_df', '_df_1')
    )
else:
    merged_df = pd.DataFrame()  # No matches found

print("\nMerged DataFrame:")
print(merged_df)

[{'df_index': 141, 'df_1_index': 17, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 3796, 'df_1_index': 27, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 722, 'df_1_index': 36, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 3286, 'df_1_index': 38, 'track_score': 70.58823529411764, 'artist_score': 100.0}, {'df_index': 226, 'df_1_index': 57, 'track_score': 71.42857142857143, 'artist_score': 100.0}, {'df_index': 335, 'df_1_index': 86, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 8786, 'df_1_index': 86, 'track_score': 85.71428571428572, 'artist_score': 100.0}, {'df_index': 1656, 'df_1_index': 130, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 1184, 'df_1_index': 137, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 1180, 'df_1_index': 161, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 4271, 'df_1_index': 170, 'track_score': 100.0, 'artist_score': 100.0}, {'df_index': 2468, 'df_1_index': 171, 'track_score

In [ ]:
merged_df.to_csv("merged_new.csv", index=False)